In [74]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sqlalchemy as db
from sqlalchemy import create_engine

In [75]:
path = 'clean_songs.csv'
db_path = 'sqlite:///spotify.db'

In [76]:
df = pd.read_csv(path)
df.drop(['Unnamed: 0', 'id'], axis=1, inplace=True)
df.index.rename('id', inplace=True)
df = pd.DataFrame(df)
df.head()

,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,energy,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date
id,,,,,,,,,,,,,,,,,,,,,
0,Downtown Life,Ooh Yeah!,4EEyCnj2fZu1eInXpLfZZW,['Daryl Hall & John Oates'],['77tT1kLj6mCWtFNqiOmP9H'],1,1,False,0.660,0.892,...,0.0419,0.0971,0.000021,0.3500,0.767,110.089,268467,4,1988,7/27/1988
1,Everything Your Heart Desires,Ooh Yeah!,4EEyCnj2fZu1eInXpLfZZW,['Daryl Hall & John Oates'],['77tT1kLj6mCWtFNqiOmP9H'],2,1,False,0.706,0.861,...,0.0411,0.1570,0.000001,0.2560,0.887,99.992,300467,4,1988,7/27/1988
2,I'm In Pieces,Ooh Yeah!,4EEyCnj2fZu1eInXpLfZZW,['Daryl Hall & John Oates'],['77tT1kLj6mCWtFNqiOmP9H'],3,1,False,0.381,0.824,...,0.0408,0.1190,0.000000,0.0826,0.805,87.064,290467,4,1988,7/27/1988
3,Missed Opportunity,Ooh Yeah!,4EEyCnj2fZu1eInXpLfZZW,['Daryl Hall & John Oates'],['77tT1kLj6mCWtFNqiOmP9H'],4,1,False,0.765,0.815,...,0.0429,0.5230,0.000000,0.0821,0.733,106.063,287600,4,1988,7/27/1988
4,Talking All Night,Ooh Yeah!,4EEyCnj2fZu1eInXpLfZZW,['Daryl Hall & John Oates'],['77tT1kLj6mCWtFNqiOmP9H'],5,1,False,0.745,0.787,...,0.0451,0.2190,0.000110,0.0910,0.902,120.925,274400,4,1988,7/27/1988


In [77]:
eighty_eight = df[df['year']== 1988]
eighty_eight.head(2)

,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,energy,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date
id,,,,,,,,,,,,,,,,,,,,,
0,Downtown Life,Ooh Yeah!,4EEyCnj2fZu1eInXpLfZZW,['Daryl Hall & John Oates'],['77tT1kLj6mCWtFNqiOmP9H'],1,1,False,0.660,0.892,...,0.0419,0.0971,0.000021,0.350,0.767,110.089,268467,4,1988,7/27/1988
1,Everything Your Heart Desires,Ooh Yeah!,4EEyCnj2fZu1eInXpLfZZW,['Daryl Hall & John Oates'],['77tT1kLj6mCWtFNqiOmP9H'],2,1,False,0.706,0.861,...,0.0411,0.1570,0.000001,0.256,0.887,99.992,300467,4,1988,7/27/1988


In [78]:
ninety_nine = df[df['year']== 1999]
ninety_nine.head(2)

,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,energy,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date
id,,,,,,,,,,,,,,,,,,,,,
3969,Testify,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],1,1,False,0.470,0.978,...,0.0727,0.0261,0.000011,0.356,0.503,117.906,210133,4,1999,11/2/1999
3970,Guerrilla Radio,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],2,1,True,0.599,0.957,...,0.1880,0.0129,0.000071,0.155,0.489,103.680,206200,4,1999,11/2/1999


In [79]:
o_nine = df[df['year']== 2009]
o_nine.head(2)

,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,energy,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date
id,,,,,,,,,,,,,,,,,,,,,
28182,Hello,Drowning By Proxy,7ao9SFaMgsAPNol7RnINa8,['Full Catastrophe Living'],['1YQWsUf74fWC9zHvFXIpmB'],1,1,False,0.523,0.351,...,0.0440,0.90500,0.935,0.0699,0.487,122.861,252000,4,2009,5/1/2009
28183,Drowning By Proxy,Drowning By Proxy,7ao9SFaMgsAPNol7RnINa8,['Full Catastrophe Living'],['1YQWsUf74fWC9zHvFXIpmB'],2,1,False,0.643,0.483,...,0.0294,0.00131,0.410,0.2870,0.884,109.129,190000,4,2009,5/1/2009


In [80]:
engine = create_engine(db_path)
conn = engine.connect()

In [81]:
attributes_df = df[['explicit', 'danceability', 'energy', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo']]

In [82]:
attributes_df.index.rename('attribute_id', inplace=True)

attributes_df.to_sql(name= 'attributes', con = engine, if_exists='replace')

In [84]:
artist_df = df[["artists"]]
artist_df.index.rename("artist_id", inplace=True)
artist_df.to_sql(name = "artist", con = engine, if_exists='replace')